In [1]:
# Install kaggle
!pip install kaggle

# Upload kaggle.json (API token from your Kaggle account settings)
from google.colab import files
files.upload()  # upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pavan8055","key":"09d87282899bbf662af18306774a3a5f"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d parisrohan/credit-score-classification
!unzip -o credit-score-classification.zip

Dataset URL: https://www.kaggle.com/datasets/parisrohan/credit-score-classification
License(s): CC0-1.0
  0% 0.00/9.51M [00:00<?, ?B/s]
100% 9.51M/9.51M [00:00<00:00, 1.11GB/s]
Archive:  credit-score-classification.zip
  inflating: test.csv                
  inflating: train.csv               


In [8]:
import pandas as pd

train_df = pd.read_csv("/content/train.csv", low_memory=False)
print(train_df.columns.tolist())


['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score']


In [ ]:
!pip install scikit-learn==1.4.2

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load data
train_df = pd.read_csv("train.csv", low_memory=False)

# Drop unnecessary columns
drop_cols = ["ID", "Customer_ID", "SSN", "Name"]
train_df = train_df.drop(columns=drop_cols, errors="ignore")

# Fill missing values
train_df = train_df.ffill()

# Convert numeric columns
numeric_cols = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts",
                "Num_Credit_Card", "Interest_Rate", "Num_of_Loan", "Delay_from_due_date",
                "Num_of_Delayed_Payment", "Outstanding_Debt", "Credit_Utilization_Ratio",
                "Total_EMI_per_month"]

for col in numeric_cols:
    train_df[col] = pd.to_numeric(train_df[col].astype(str).str.replace(r'[^0-9.-]', '', regex=True), errors='coerce')

train_df = train_df.fillna(train_df.median(numeric_only=True))

# Encode categorical columns
label_encoders = {}
for col in train_df.select_dtypes(include="object").columns:
    if col == "Credit_Score":
        continue
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    label_encoders[col] = le

# Encode target
target_le = LabelEncoder()
y = target_le.fit_transform(train_df["Credit_Score"])
X = train_df.drop(columns=["Credit_Score"])

# Train RandomForest
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X, y)

# Save model, encoders, target encoder, and feature columns
feature_cols = X.columns.tolist()
joblib.dump((model, label_encoders, target_le, feature_cols), "credit_model.pkl")
print("✅ Model saved with feature columns")


✅ Model saved with feature columns
